# PRÁCTICA 10: PERCEPTRÓN MULTICAPA

###### Técnicas de Aprendizaje Automático 2019/2020 <br>Patricia Aguado Labrador

In [1]:
from sklearn.datasets import fetch_openml
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score as ac
import time

#### Descripción del conjunto de datos:

Dataset perteneciente al dominio de la enfermedad relacionada con la tiroides. Está formado por 26 atributos numéricos y una clase nominal que puede tomar cinco valores en el rango ['1','5'].

In [2]:
data = fetch_openml(name='thyroid-allhypo')

In [3]:
X = data.data
# Normalizar
X = MinMaxScaler().fit_transform(X)

y = data.target
X.shape, y.shape

((2800, 26), (2800,))

### MLP clasificador con validación cruzada (k=10)

In [4]:
# Creación de 10 particiones para realizar validación cruzada
skf = StratifiedKFold(n_splits=10, random_state=0, shuffle=True)
# Array para almacenar los resultados por tamaño de capa oculta
results_summary = np.zeros((10,3), dtype='float')
size = 0
for i in range(10,110,10):
    # Array para almacenar los resultados de las 10 particiones de cross-validation para cada tamaño de capa oculta
    results_per_size = np.zeros((10,3), dtype='float')
    k = 0
    print("\033[1mTamaño de capa:\033[0m",i)
    
    # Creación, entrenamiento y predicción del clasificador MLP para las 10 particiones
    for train_index, test_index in skf.split(X,y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        start = time.time()
        
        clf = MLPClassifier(hidden_layer_sizes=(i,), max_iter=2000, random_state=0)
        clf.fit(X_train, y_train)
        
        predict_train = clf.predict(X_train)
        predict_test = clf.predict(X_test)
        
        # Tasa de precisión en entrenamiento y en test
        ac_train = ac(y_train, predict_train)
        ac_test = ac(y_test, predict_test)
        
        end = time.time()
        execution_time = end - start
        
        results_per_size[k] = [ac_train,ac_test,execution_time]
        k += 1
    
    # Impresión de resultados para cada tamaño de capa oculta
    print('\33[4mFold     Entrena     Verifica     T.Ejecución\33[0m')
    for i in range(10):
        print(i,'\t %.4f'%results_per_size[i][0],'     %.4f'%results_per_size[i][1],'      %.4f'%results_per_size[i][2])   
    print('\n')
    
    # Resumen de los resultados a través de la media de las precisiones y de la suma total del tiempo empleado
    mean_train = np.mean(results_per_size[:,0], axis = 0, dtype=np.float64)
    mean_test = np.mean(results_per_size[:,1], axis = 0, dtype=np.float64)
    mean_execution_time = np.sum(results_per_size[:,2], axis = 0, dtype=np.float64)
    
    # Impresión del resumen de resultados
    print('MEDIA (Entrena)= %.4f' %mean_train)
    print('MEDIA (Verifica)= %.4f' %mean_test)
    print('Tiempo Total (seg)= %.4f' %mean_execution_time)
    print('\n')
    
    results_summary[size] = [mean_train, mean_test, mean_execution_time]
    size += 1

Tamaño de capa: 10
Fold     Entrena     Verifica     T.Ejecución
0 	 0.7242      0.7148       3.0782
1 	 0.7225      0.7402       2.8689
2 	 0.7310      0.6786       3.0221
3 	 0.7159      0.7750       3.1784
4 	 0.7258      0.6893       2.7293
5 	 0.7251      0.7133       3.3029
6 	 0.7279      0.7168       4.3273
7 	 0.7259      0.7025       3.8857
8 	 0.7263      0.6667       2.9185
9 	 0.7196      0.7384       2.4553


MEDIA (Entrena)= 0.7244
MEDIA (Verifica)= 0.7135
Tiempo Total (seg)= 31.7666


Tamaño de capa: 20
Fold     Entrena     Verifica     T.Ejecución
0 	 0.7254      0.7148       1.9636
1 	 0.7265      0.7402       2.4971
2 	 0.7266      0.6679       2.1070
3 	 0.7171      0.7643       1.9386
4 	 0.7302      0.6821       2.9887
5 	 0.7295      0.7097       2.8861
6 	 0.7247      0.7240       2.6783
7 	 0.7279      0.7025       2.3669
8 	 0.7338      0.6631       2.8107
9 	 0.7203      0.7312       2.1287


MEDIA (Entrena)= 0.7262
MEDIA (Verifica)= 0.7100
Tiempo Total (seg)

### Resumen de resultados

In [5]:
# Impresión de los resultados para cada tamaño de capa oculta
size = 10
print('\33[4mNo.Ocultas     Entrena     Verifica     T.Ejecución\33[0m')
for i in range(10):
    print(size,'\t       %.4f'%results_summary[i][0],'     %.4f'%results_summary[i][1],'      %.4f'%results_summary[i][2])
    size +=10

No.Ocultas     Entrena     Verifica     T.Ejecución
10 	       0.7244      0.7135       31.7666
20 	       0.7262      0.7100       24.3656
30 	       0.7307      0.7089       27.0310
40 	       0.7283      0.7117       25.2900
50 	       0.7333      0.7086       30.6974
60 	       0.7327      0.7085       28.4695
70 	       0.7319      0.7128       28.3958
80 	       0.7302      0.7068       29.5684
90 	       0.7316      0.7128       35.7733
100 	       0.7337      0.7064       34.2972


In [8]:
# Mayor precisión media en el test
best_ac = np.argmax(results_summary[:,1], axis=0)
print("Entendiendo mejor resultado como la mayor precisión media en la prueba del clasificador, obtenemos este con un tamaño de capa oculta de:",(best_ac+1)*10,'(Verifica: %.4f)'%results_summary[best_ac][1])

Entendiendo mejor resultado como la mayor precisión media en la prueba del clasificador, obtenemos este con un tamaño de capa oculta de: 10 (Verifica: 0.7135)


In [9]:
# Menor tiempo de ejecución
best_t = np.argmin(results_summary[:,2], axis=0)
print("El resultado que menor tiempo conlleva de ejecución lo obtenemos con un tamaño de capa de:",(best_t+1)*10, '(T.Ejecución: %.4f)'%results_summary[best_t][2])

El resultado que menor tiempo conlleva de ejecución lo obtenemos con un tamaño de capa de: 20 (T.Ejecución: 24.3656)
